---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [3]:
def get_home_price_df():
    home_price_df = pd.read_csv("City_Zhvi_AllHomes.csv")
    return home_price_df

def get_list_of_university_towns():
    col_headings=['State', 'RegionName']
    university_towns_df = pd.DataFrame(columns=col_headings)
    
    fhandle = open('university_towns.txt')
    state_identifer = '[edit]'
    state = None
    region_name = None
    lst = []
    for line in fhandle:
        clean_line = line.strip()
        if line == '' : break
        if state_identifer in clean_line:
            state = clean_line.replace(state_identifer, '')
            region_name = None
            continue
        words = clean_line.split('(')
        region_name = words[0].strip()
        lst.append([state, region_name])
    
    col_headings=['State', 'RegionName']
    university_towns_df = pd.DataFrame(lst, columns=col_headings)

    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return university_towns_df

In [4]:
def get_GDP_df():
    col_headings=['Year and Qtr','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
    GDP_df = pd.read_excel('gdplev.xls', sheetname='Sheet1', skiprows=7, 
                          usecols=[4,5,6], names=col_headings)
    return GDP_df

def split_year_qtr(date_string):
    year = None
    qtr = None
    date_items = date_string.split('q')
    year = int(date_items[0])
    qtr = int(date_items[1])
    return year, qtr

def get_recession_start():
    GDP = get_GDP_df()
    
    first_qtr = None
    second_qtr = None
    third_qtr = None
    year = None
    recession_start = None
    
    for i in range(0, len(GDP)-2):
        first_qtr = GDP.iloc[i]['GDP in billions of chained 2009 dollars']
        second_qtr = GDP.iloc[i+1]['GDP in billions of chained 2009 dollars']
        third_qtr = GDP.iloc[i+2]['GDP in billions of chained 2009 dollars']
        year = int(GDP.iloc[i]['Year and Qtr'][:-2])

        if (first_qtr > second_qtr) & (second_qtr > third_qtr) & (year >= 2000):
            recession_start = str(GDP.iloc[i+1]['Year and Qtr'])
            break 

    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return recession_start

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    GDP = get_GDP_df()
    
    rec_start_year, rec_start_qtr = split_year_qtr(get_recession_start())
    
    first_qtr = None
    second_qtr = None
    third_qtr = None
    year = None
    rec_end = None
    
    for i in range(0, len(GDP)-2):
        first_qtr = GDP.iloc[i]['GDP in billions of chained 2009 dollars']
        second_qtr = GDP.iloc[i+1]['GDP in billions of chained 2009 dollars']
        third_qtr = GDP.iloc[i+2]['GDP in billions of chained 2009 dollars']

        year, qtr = split_year_qtr(GDP.iloc[i]['Year and Qtr'])

        if ((second_qtr > first_qtr) & (third_qtr > second_qtr) & (year >= 2000) 
            & (year >= rec_start_year) & (qtr >= rec_start_qtr)):
            rec_end = str(GDP.iloc[i+2]['Year and Qtr'])
            break
       
    return rec_end

In [6]:
def get_recession_bottom():
    
    GDP = get_GDP_df().copy()
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    
    rec_start_ix = GDP.loc[GDP['Year and Qtr'] == rec_start].index.item()
    rec_end_ix = GDP.loc[GDP['Year and Qtr'] == rec_end].index.item()
    GDP = GDP.iloc[rec_start_ix:rec_end_ix]
    rec_bottom_ix = GDP[GDP['GDP in billions of chained 2009 dollars'] 
                         == (GDP['GDP in billions of chained 2009 dollars'].min())].index.item()
    rec_bottom = GDP.get_value(rec_bottom_ix, 'Year and Qtr')
                  
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return rec_bottom

In [7]:
def convert_housing_data_to_quarters():
    
    home_price_df = pd.read_csv("City_Zhvi_AllHomes.csv")
    col_names1 = ['RegionName', 'State']
    col_names2 = []
    year = 2000
    month = 1
    while year < 2017:
        col_name_year = year
        while month < 13:
            if (month < 10):
                col_name_month = '0' + str(month)
            else:
                col_name_month = str(month)    
            col_name = str(col_name_year) + '-' + str(col_name_month)
            col_names2.append(col_name)   
            if (month == 12):
                month = 1
                break
            if (month == 8) & (year == 2016):
                break
            month = month + 1
        year = year + 1 
        if (year == 2017):
            break 
            
    col_names1 = col_names1 + col_names2
    home_price_df = pd.read_csv("City_Zhvi_AllHomes.csv", header=0)
    home_price_df = home_price_df[col_names1]
    
    col_names3 = []
    year = 2000
    qtr = 1
    while year < 2017:
        col_name_year = year
        while qtr < 5: 
            col_name = str(col_name_year) + 'q' + str(qtr)
            col_names3.append(col_name)
            if (qtr == 4):
                qtr = 1
                break
            if (qtr == 3) & (year == 2016):
                break    
            qtr = qtr + 1
        year = year + 1
        if (year == 2017):
            break 
            
    i=0
    for period in col_names3:
        col_name = []
        try:       
            col_name = col_names2[i:i+3]
        except:
            col_name = col_names2[i:i+2]   
        home_price_df[period] = home_price_df[col_name].mean(axis=1, skipna=True)
        i = i+3

    home_price_df = home_price_df.drop(col_names2, axis=1)
    
    for state in states:
        row = home_price_df[home_price_df['State'] == state]
        home_price_df = home_price_df.replace(to_replace=state, value=states[state], regex=False)
        
    home_price_df.set_index(['State','RegionName'], inplace=True)
    
    '''
    Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    return home_price_df

In [8]:
def run_ttest():
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    year, qtr = split_year_qtr(rec_start)
    if (qtr == 1):
        qtr = 4
        year = year-1
    else: 
        qtr = qtr-1
    rec_start_qtr_before = str(year) + 'q' + str(qtr)
    
    house_prices = convert_housing_data_to_quarters().copy()
    university_towns = get_list_of_university_towns().copy()
    university_towns.set_index(['State','RegionName'], inplace=True)
    
    house_prices = house_prices.loc[:,rec_start_qtr_before:rec_bottom]
    
    house_prices['Uni_Town'] = 'No'
    house_prices['PriceRatio'] = house_prices[rec_start_qtr_before].div(house_prices[rec_bottom])
    house_prices = house_prices.sort_index()
    
    uni_towns_lst = []
    uni_towns_lst = university_towns.index.values.tolist()
    
    for town in uni_towns_lst:
        try:
            '''
            print(house_prices.loc[town])
            print('has value')
            print('------------------------------------')
            '''
            house_prices.loc[town, 'Uni_Town'] = 'Yes'
            '''
            print(house_prices.loc[town])
            print('after update')
            print('------------------------------------')
            '''
        except:
            continue
            #print(town)
            #print('no value')
        #print('••••••••••••••••••••••••••••••••••••••')
    
    uni_towns = house_prices[house_prices['Uni_Town'] == 'Yes']
    not_uni_towns = house_prices[house_prices['Uni_Town'] == 'No']
    
   
    uni_towns_ratio = uni_towns['PriceRatio'].mean()
    none_uni_towns_ratio = not_uni_towns['PriceRatio'].mean()
    if (uni_towns_ratio < none_uni_towns_ratio):
        better = 'university town'
    else:
        better = 'non-university town'
    
    t, p = ttest_ind(not_uni_towns['PriceRatio'], uni_towns['PriceRatio'], nan_policy='omit')
    
    if (p<0.1):
        different = True
    else:
        different = False

    result = (different, p, better)
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return result